In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time

In [2]:
# Reading all templates needed
intensities_template = pd.read_csv("intensities_template.csv")
samples_template = pd.read_csv("samples_template.csv", dtype={"MOUSE": "string","SAMPLE": int,"FILES_HD": int,"FILES_BOX": int,"HEMI": "string","STAIN": "string","TX_GROUP": "string", "GENDER":"string", "CONTEXT":"string"})

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
#Counting number of files on folder
list = os.listdir('SMM_RH_tdT_non_social_CSVs_DR') 
number_files = len(list)

# Changing directory to the folder
os.chdir(r'C:\Users\greci\Documents\Jupiter_notebooks\SMM_RH_tdT_non_social_CSVs_DR')

number_files

17

In [5]:
#Using and timing  a loop to process all files and populate the main data-frame


start = time.time()

appended_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    # Saving sample's number and reviewer's name
    file = list[i]
    sample = file[5:7]


    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    ##########################################################################################################################
    
    # store DataFrame in list
    appended_data.append(output)
    
    i += 1
    
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
end = time.time()


print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

appended_data


C:\Users\greci\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


17  Files processed 
 
 Execution time: 7.7 seconds


,LabelID,Region,LabelAbrv,ACTIVITY,SAMPLE
0,1,Right Tuberomammillary nucleus ventral part,RTMv,24.0,1.0
1,2,Right Primary somatosensory area mouth layer 6b,RSSp-m6b,33.0,1.0
2,4,Right Inferior colliculus,RIC,4709.0,1.0
3,6,Right internal capsule,Rint,315.0,1.0
4,7,Right Principal sensory nucleus of the trigeminal,RPSV,234.0,1.0
...,...,...,...,...,...
1106,21127,Left Temporal association areas layer 2/3,LTEa2/3,NaN,35.0
1107,21128,Left Primary somatosensory area lower limb lay...,LSSp-ll5,NaN,35.0
1108,21139,Left Nucleus of the lateral olfactory tract la...,LNLOT3,NaN,35.0
1109,30703,"Left Dentate gyrus, molecular layer",LDG-mo,34.0,35.0


In [6]:
samples_template

,MOUSE,SAMPLE,FILES_HD,FILES__BOX,HEMI,STAIN,TX_GROUP,GENDER,CONTEXT
0,1001a,1,3778,3778,R,anti-tdT,Saline,F,single housed
1,1001a,2,3632,3632,L,anti-cfos,Saline,F,group housed
2,1002a,3,4074,4074,R,anti-tdT,MDMA,F,single housed
3,1002a,4,3738,3738,L,anti-cfos,MDMA,F,group housed
4,1003a,5,3854,3854,R,anti-tdT,Saline,M,single housed
5,1003a,6,3870,3870,L,anti-cfos,Saline,M,group housed
6,1004a,7,3820,3820,R,anti-tdT,MDMA,M,single housed
7,1004a,8,3612,3612,L,anti-cfos,MDMA,M,group housed
8,1005a,9,3840,3840,R,anti-tdT,METH,F,single housed
9,1005a,10,4056,4056,L,anti-cfos,METH,F,group housed


In [23]:
test = pd.pivot_table(appended_data, values='ACTIVITY', index=['SAMPLE'],
                    columns=['LabelAbrv'], aggfunc=np.sum, dropna = True).reset_index('SAMPLE')
test

LabelAbrv,SAMPLE,LAAA,LACAd1,LACAd2/3,LACAd5,LACAd6a,LACAd6b,LACAv1,LACAv2/3,LACAv5,...,Rst,Rtb,Rts,Rtsp,RvVIIIn,Rvhc,Rvon,Rvtd,Rx,Ry
0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,...,203.0,79.0,6.0,51.0,59.0,13.0,0.0,40.0,23.0,23.0
1,3.0,0.0,179.0,77.0,2.0,0.0,0.0,468.0,113.0,3.0,...,444.0,82.0,0.0,56.0,73.0,85.0,0.0,35.0,11.0,3.0
2,5.0,0.0,38.0,8.0,0.0,0.0,0.0,85.0,13.0,0.0,...,160.0,126.0,8.0,62.0,100.0,44.0,0.0,19.0,35.0,16.0
3,7.0,0.0,205.0,142.0,0.0,0.0,0.0,446.0,169.0,2.0,...,315.0,89.0,20.0,80.0,204.0,32.0,0.0,97.0,31.0,19.0
4,9.0,0.0,51.0,12.0,0.0,0.0,0.0,15.0,1.0,0.0,...,458.0,126.0,14.0,94.0,100.0,13.0,0.0,13.0,58.0,49.0
5,11.0,0.0,71.0,46.0,0.0,0.0,0.0,153.0,40.0,2.0,...,182.0,83.0,10.0,92.0,50.0,38.0,0.0,41.0,9.0,29.0
6,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,219.0,96.0,0.0,19.0,50.0,6.0,0.0,0.0,21.0,33.0
7,15.0,0.0,15.0,2.0,0.0,0.0,0.0,22.0,6.0,0.0,...,359.0,104.0,0.0,80.0,55.0,62.0,0.0,30.0,49.0,27.0
8,17.0,0.0,6.0,0.0,0.0,0.0,0.0,121.0,10.0,0.0,...,369.0,113.0,0.0,68.0,80.0,64.0,0.0,73.0,20.0,2.0
9,19.0,0.0,392.0,253.0,14.0,0.0,0.0,593.0,280.0,13.0,...,326.0,60.0,2.0,136.0,26.0,104.0,0.0,62.0,29.0,1.0


In [33]:
all_data = pd.merge(test,samples_template[['SAMPLE','GENDER','TX_GROUP','CONTEXT']],on='SAMPLE',how='outer')
all_data = all_data.dropna(subset=['LAAA'])
#all_data = all_data.dropna(axis=0, subset=['CONTEXT'])
all_data

,SAMPLE,LAAA,LACAd1,LACAd2/3,LACAd5,LACAd6a,LACAd6b,LACAv1,LACAv2/3,LACAv5,...,Rtsp,RvVIIIn,Rvhc,Rvon,Rvtd,Rx,Ry,GENDER,TX_GROUP,CONTEXT
0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,...,51.0,59.0,13.0,0.0,40.0,23.0,23.0,F,Saline,single housed
1,3.0,0.0,179.0,77.0,2.0,0.0,0.0,468.0,113.0,3.0,...,56.0,73.0,85.0,0.0,35.0,11.0,3.0,F,MDMA,single housed
2,5.0,0.0,38.0,8.0,0.0,0.0,0.0,85.0,13.0,0.0,...,62.0,100.0,44.0,0.0,19.0,35.0,16.0,M,Saline,single housed
3,7.0,0.0,205.0,142.0,0.0,0.0,0.0,446.0,169.0,2.0,...,80.0,204.0,32.0,0.0,97.0,31.0,19.0,M,MDMA,single housed
4,9.0,0.0,51.0,12.0,0.0,0.0,0.0,15.0,1.0,0.0,...,94.0,100.0,13.0,0.0,13.0,58.0,49.0,F,METH,single housed
5,11.0,0.0,71.0,46.0,0.0,0.0,0.0,153.0,40.0,2.0,...,92.0,50.0,38.0,0.0,41.0,9.0,29.0,M,METH,single housed
6,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.0,50.0,6.0,0.0,0.0,21.0,33.0,F,Saline,single housed
7,15.0,0.0,15.0,2.0,0.0,0.0,0.0,22.0,6.0,0.0,...,80.0,55.0,62.0,0.0,30.0,49.0,27.0,F,MDMA,single housed
8,17.0,0.0,6.0,0.0,0.0,0.0,0.0,121.0,10.0,0.0,...,68.0,80.0,64.0,0.0,73.0,20.0,2.0,M,MDMA,single housed
9,19.0,0.0,392.0,253.0,14.0,0.0,0.0,593.0,280.0,13.0,...,136.0,26.0,104.0,0.0,62.0,29.0,1.0,F,Saline,single housed


In [38]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [40]:
X = all_data.drop(columns=["SAMPLE","TX_GROUP","CONTEXT","GENDER"])
y = all_data[["TX_GROUP"]]

In [46]:
clf = LogisticRegression(solver='liblinear').fit(X, y.values.ravel())

In [49]:
clf.get_params(deep=True)

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [53]:
clf.fit(X,y.values.ravel())
predictions = clf.predict(X)

In [54]:
from sklearn.metrics import classification_report
print(classification_report(y.values.ravel(),predictions))

              precision    recall  f1-score   support

        MDMA       1.00      1.00      1.00         7
        METH       1.00      1.00      1.00         4
      Saline       1.00      1.00      1.00         6

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



In [55]:
appended_data

,LabelID,Region,LabelAbrv,ACTIVITY,SAMPLE
0,1,Right Tuberomammillary nucleus ventral part,RTMv,24.0,1.0
1,2,Right Primary somatosensory area mouth layer 6b,RSSp-m6b,33.0,1.0
2,4,Right Inferior colliculus,RIC,4709.0,1.0
3,6,Right internal capsule,Rint,315.0,1.0
4,7,Right Principal sensory nucleus of the trigeminal,RPSV,234.0,1.0
...,...,...,...,...,...
1106,21127,Left Temporal association areas layer 2/3,LTEa2/3,NaN,35.0
1107,21128,Left Primary somatosensory area lower limb lay...,LSSp-ll5,NaN,35.0
1108,21139,Left Nucleus of the lateral olfactory tract la...,LNLOT3,NaN,35.0
1109,30703,"Left Dentate gyrus, molecular layer",LDG-mo,34.0,35.0


In [ ]:
# Reading pixels files to calculte volumes

In [ ]:
header = ["PIXEL_COUNT"]
pixels_count_volume = pd.read_csv("SMM_s1_ABA_histogram_total.csv", names = header)
pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume)))  
pixels_count_volume_in_tissue = pd.read_csv("SMM_s1_ABA_histogram_in_tissue_total.csv", names = header)
pixels_count_volume_in_tissue.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume_in_tissue)))  

In [ ]:
# Reading pixels file to count cell activity

In [ ]:
regions_count_activity_fracc = pd.read_csv("SMM_s1_DR_Seg_3Dcount_75slices.csv")
regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"] 

In [ ]:
# Adding-up regions' fracctions

In [ ]:
regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
total_region_activity = regions_fracc_1.append(regions_fracc_2, ignore_index=True).append(regions_fracc_3, ignore_index=True)
total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)

In [ ]:
# Adding volumes and activity columns to the template table

In [ ]:
output_s1 = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
output_s1 = output_s1.drop(columns=['INTENSITY'])
output_s1 = output_s1.rename(columns = {'PIXEL_COUNT':'VOLUME'})
output_s1 = output_s1.merge(pixels_count_volume_in_tissue, how='left', left_on='LabelID', right_on='INTENSITY')
output_s1['VOLUME'] = output_s1['VOLUME'].div(1000000).round(6)
output_s1 = output_s1.drop(columns=['INTENSITY'])
output_s1 = output_s1.rename(columns = {'PIXEL_COUNT':'VOLUME_IN_TISSUE'})
output_s1['VOLUME_IN_TISSUE'] = output_s1['VOLUME_IN_TISSUE'].div(1000000).round(6)
output_s1["LabelID"] = output_s1["LabelID"].astype(int)


total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)
output_s1 = output_s1.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
output_s1 = output_s1.drop(columns=['INTENSITY'])
output_s1 = output_s1.rename(columns = {'COUNTS':'ACTIVITY'})

In [ ]:
output_s1.to_csv(r's1_DR.csv')